<p style="float: left;"><a href="_index.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="p4-exercises.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Actor model

The **actor model** is a mathematical framework for managing parallel computation in high-performance networks.

It specifically tackles challenges such as:

* Encapsulation,
* The illusion of a call stack.

These challenges arise because **traditional programming assumptions no longer align with the realities of modern multi-threaded, multi-CPU systems.**

## The Problem of distributed domputation and object-oriented programming (OOP)

- **One of the OPP core pillars  is _encapsulation_.**
  
- Encapsulation dictates that **the internal data of an object is not accessible directly from the outside**; it can only be modified by invoking a set of curated methods. 

- The object is responsible for exposing safe operations that protect the invariant nature of its encapsulated data.

    ![](https://getakka.net/images/seq_chart.png)
    _Runtime behaviour. Interactions of method calls._

    - **A _thread_ executes all the method's calls, and the enforcement of invariants occurs on the same thread**
    from which the method was called.
    
        ![](https://getakka.net/images/seq_chart_thread.png)
        _Executions accurs on the same thread._

    - **In multiple thread system, the invariance might not be respected.**

      ![](https://doc.akka.io/libraries/akka-core/current/typed/guide/diagrams/seq_chart_multi_thread.png)
        _Method's executions in a multi-thred system._

        - **The encapsulation model of objects does not guarantee anything about what happens where two threads enter the same method**.
                    
        - **The problem appears because objects share mutable data between them (data raise) and <span style="color: red">you are not suppose to abstract over data raises**.</span>

        - Instructions of the two invocations can be interleaved in arbitrary ways which eliminate **any hope for keeping the invariants intact without some type of coordination.**

        - A common approach to solving this problem is to add a lock around these methods.

            - Locks are very costly on modern CPU architectures, requiring heavy-lifting from the operating system to suspend the thread and restore it later.

            - The caller thread is now blocked, so it cannot do any other meaningful work.
    
            - Locks introduce a new menace: deadlocks.

            - When it comes to coordinating across multiple machines, the only alternative is distributed locks. <span style="color:red">**Distributed locks are several magnitudes less efficient than local locks**. </span>

## Call stacks and tasks delegation

- **A call stack is a stack data structure that stores information about the active subroutines of a computer program.**
      
- Call stacks seems to be a good way to operate between threds.


- **They don’t handle asynchronous operations well because they don’t cross thread boundaries <span style="color:red">(isolated)</span>**

    - **Completion notification**: How does the "caller" thread know the task is complete in the "worker" thread?
 
    - **Exception handling**: If an error occurs, the exception goes to the worker’s handler, bypassing the caller. Without a direct way to notify the caller.
 
    - Messages can be lost. This leaves the system without a way to recover the lost task, even in local (non-networked) communication.
         
      ![](https://getakka.net/images/exception_prop.png)

-  <span style="color:red">Concurrent systems must handle service faults and provide recovery mechanisms.</span>


## Actors for distributed systems

### What is an actor?

- An actor as the basic **building block of concurrent computation.**
  
- **It encapsulates computation and holds its own private state**
  
- **Actors communicates with other actors solely by passing asynchronous messages.**
  
- **Actors have a mailbox where messages arrive and are processed at most one at a time** in the order they are received.
  
- **How an actor responds / reacts to a message essentially defines its behavior.**

- Essenciatily, an actor can:
  
    1) Send messages to other actors,
       
    2) Respond to received messages,
       
    3) Create new actors.


![](https://getakka.net/images/serialized_timeline_invariants.png) _Exemple of asynchronous messages between actors_

### Actor's hierarchical structure

- **Actors are hierarchy organized.**

    - An actor responsible for a particular function might choose to break down into smaller manageable segments.

    - Creating child actors, over which it maintains control.
 
- **Each actor has one supervisor, the actor that initiated its creation.**
  
- **When an actor is unable to address a certain situation, it sends a failure message to its supervisor**, requesting assistance.

- **This recursive setup enables failure handling at the appropriate level in the hierarchy.**

![](https://getakka.net/images/actor_tree_supervision.png) _Actor's hierarchy. Failure handling_

## Actors & SCALA


- We will use the [Akka actors](https://doc.akka.io/libraries/akka-core/current/typed/actors.html#akka-actors) of the [Akka platform](https://github.com/akka/akka).

- Akka’s actor model provides an abstraction that simplifies writing correct concurrent, parallel, and distributed systems.

- The Akka platform offers:

  - **Akka actors** – toolkit for building concurrent and distributed applications
    
  - **Concurrency** – safe and efficient message-driven execution
    
  - **Scalability** – scale locally with threads or across nodes with remote actors
    
  - **Fault tolerance** – supervision and self-healing mechanisms
    
  - **Unified programming model** – same abstraction for local and distributed actors
    
  - **Integrated runtime management** – monitoring and deployment tools
    
  - **Open-source** – community-driven and widely adopted
 
- <span style="color:red">**We will use the object-oriented style.**</span> **Where a concrete class for the actor behavior is defined and mutable state is kept inside of it as fields.**

### Receiving Messages

In classic Akka, each actor’s behavior is defined by a type called `Receive`, which is essentially specified by the partial method `def receive`.
Summaring, actors receive messages through the `receive` method and the behaviour defined per message in that method defines, what the actors do.

In [1]:
import $ivy.`com.typesafe.akka::akka-actor:2.6.20`
import akka.actor.{Actor, ActorRef}
import akka.actor.{ActorSystem, PoisonPill, Props}
import akka.actor.{Actor, ActorRef, ActorSystem, PoisonPill, Props, Terminated}

Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-actor_3/2.6.20/akka-actor_3-2.6.20.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-java8-compat_3/1.0.0/scala-java8-compat_3-1.0.0.pom


import $ivy.$                                     

import akka.actor.{Actor, ActorRef}

import akka.actor.{ActorSystem, PoisonPill, Props}

import akka.actor.{Actor, ActorRef, ActorSystem, PoisonPill, Props, Terminated}


In [1]:
scala.util.Properties.versionString

res1: String = "version 2.13.16"

### Messages and Immutability

Messages can be of any kind of object but they have to be immutable. Messages are sent to an Actor using one of the following methods:

* **tell (!)** : a "fire-and-forget" approach, where a message is sent asynchronously, and the sender returns immediately without waiting for a response.

* **ask (?)** : sends a message asynchronously but returns a `Future` that represents a possible reply from the receiver.

In each of these methods, the sender has the option to include its own `ActorRef`.
This allows the receiving actor to respond directly to the sender, as the reference to the sender is passed along with the message.


## Let's play with Actors

In [1]:
import $ivy.`com.typesafe.akka::akka-actor:2.6.10`

Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-actor_2.13/2.6.10/akka-actor_2.13-2.6.10.pom
Downloaded https://repo1.maven.org/maven2/com/typesafe/config/1.4.0/config-1.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-java8-compat_2.13/0.9.0/scala-java8-compat_2.13-0.9.0.pom


import $ivy.$

In [ ]:
import $ivy.`com.typesafe.akka::akka-actor:2.6.10`
import akka.actor.{Actor, ActorRef}
import akka.actor.{ActorSystem, PoisonPill, Props}
import akka.actor.{Actor, ActorRef, ActorSystem, PoisonPill, Props, Terminated}

Failed to download https://repo1.maven.org/maven2/com/typesafe/akka/akka-actor_3/2.6.10/akka-actor_3-2.6.10.pom
Failed to download https://repo1.maven.org/maven2/com/typesafe/akka/akka-actor_3/2.6.10/akka-actor_3-2.6.10.pom.sha1
Failed to download https://jitpack.io/com/typesafe/akka/akka-actor_3/2.6.10/akka-actor_3-2.6.10.pom
Failed to download https://jitpack.io/com/typesafe/akka/akka-actor_3/2.6.10/akka-actor_3-2.6.10.pom.sha1
Failed to resolve ivy dependencies:Error downloading com.typesafe.akka:akka-actor_3:2.6.10
  not found: /home/jovyan/.ivy2/local/com.typesafe.akka/akka-actor_3/2.6.10/ivys/ivy.xml
  not found: https://repo1.maven.org/maven2/com/typesafe/akka/akka-actor_3/2.6.10/akka-actor_3-2.6.10.pom
  not found: https://jitpack.io/com/typesafe/akka/akka-actor_3/2.6.10/akka-actor_3-2.6.10.pom

In [1]:
import akka.actor.typed.Behavior
import akka.actor.typed.scaladsl.Behaviors

-- [E006] Not Found Error: cmd2.sc:1:7 -----------------------------------------
1 |import akka.actor.typed.Behavior
  |       ^^^^
  |       Not found: akka
  |
  | longer explanation available when compiling with `-explain`
-- [E006] Not Found Error: cmd2.sc:2:7 -----------------------------------------
2 |import akka.actor.typed.scaladsl.Behaviors
  |       ^^^^
  |       Not found: akka
  |
  | longer explanation available when compiling with `-explain`
Compilation Failed

In [2]:
case class Greet(name: String)
case class Praise(name: String)
case class Celebrate(name: String, age: Int)

class Talker extends Actor {
    var messages = 0
    
    def receive = {
        case Greet(name)          => greet(name)
        case Praise(name)         => praise(name)
        case Celebrate(name, age) => celebrate(name, age)
    }

    def updateMessages(n: Int) = {
        messages += n
        println(s"Message n: $messages")
    }

    def greet(name: String) = {
        updateMessages(1)
        println(s"Hello $name")
    }

    def praise(name: String) = {
        updateMessages(1)
        println(s"$name, you're amazing")
    }

    def celebrate(name: String, age: Int) = {
        updateMessages(1)
        println(s"Here's to another $age years, $name")
    }
}

defined class Greet
defined class Praise
defined class Celebrate
defined class Talker

In [3]:
class Master extends Actor {
    val talker: ActorRef = context.actorOf(Props(new Talker), "talker")
    override def preStart(): Unit =  {
        context.watch(talker)
        talker ! Greet("Huey")
        talker ! Praise("Dewey")
        talker ! Celebrate("Louie", 16)
    }

    def receive: Receive = {
        case Terminated(`talker`)  => context.system.terminate() // terminate the system
    }
}

defined class Master

In [5]:
object HelloActors {
    val system = ActorSystem("HelloActors")
    val master = system.actorOf(Props(new Master), "master")

    system.terminate()
}

HelloActors

Message n: 1
Hello Huey
Message n: 2
Dewey, you're amazing
Message n: 3
Here's to another 16 years, Louie


defined object HelloActors
res5_1: HelloActors.type = ammonite.$sess.cmd5$Helper$HelloActors$@42be562

Notice that with `actorOf` we have a reference to an `ActorRef` and not to an instance of a `Talker` class.

In [3]:
object BadHelloActors {
    val system = ActorSystem("HelloActors")
    val talker = system.actorOf(Props(new Talker), "talker")
    talker.greet()
    Thread.sleep(10000)
    system.terminate()
}

BadHelloActors

cmd4.sc:4: value greet is not a member of akka.actor.ActorRef
    talker.greet()
           ^
Compilation Failed

**NOTE:** The example below may seem misleading because this notebook is running on a single-thread container, giving the impression that everything is executing synchronously.

Let’s dive into the [actors-nightmare](https://github.com/wilberquito/actors-nightmare) repository and get it running on our PCs!

<p style="float: left;"><a href="_index.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="p4-exercises.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>